# RAG简单工作流

## 环境

python 3.12  (>3.10就好)
torch == 2.6.0 (建议)
transformers >= 4.52.0 (建议)
hnswlib 

## 文本数据格式

json列表格式, 如下：

[
    str1,
    str2,
    str3,
    ...
]

## 问答输入

格式也是可以是`list[str]`，也可以是`str`，当是`list[str]`时，默认推理模型的`batch_size`为4

## 推理输出

`list[dict[str,Any]]`

[
    {
        'query': <输入进来的问题>,
        'response': <最终模型输出的答案>,
        'search_result':<模型检索向量库时产生的结果>
    },
    ...
]


## 指令模板样式

"请依据所给的问题以及文本材料，生成最合适的答案。\n\n问题:\n{query}\n\n文本材料:\n{evidences}\n\n"

In [ ]:
import os, re, argparse
# os.environ["CUDA_VISIBLE_DEVICES"] = "1" # 这里替换成你想要的GPU序号

from Components import *

# 配置模型参数
config=dict(
    embedding_model_or_path="J:\\我的博士论文\\LLMs\\BAAI\\bge-m3",  # 你自己的文本嵌入模型地址
    embedding_text_source="data\\Text.json",       # 用于检索的文本数据源
    embedding_gpu="cuda:0",   # 其实如果初次嵌入好了之后就没必要再用GPU了，相似度计算什么的都可以跑在CPU上
    embedding_batch_size=16,   # 模型进行嵌入操作的时候的批次大小
    inference_model_or_path="J:\\我的博士论文\\LLMs\\Qwen\\Qwen2.5-Coder-7B-Instruct",  # 你自己的推理模型地址
    inference_gpu="cuda:1",   # 如果使用巨TM大的LLM推理，这里直接填`cuda`就好 （如果是其他环境就按照其他环境的参数写，比如华为的填写`npu`）
    QInt4=False,             # 是否使用QInt4量化（使用的话用于减少显存占用）
    lora_config=dict(
        path="<your_lora_path>",
        lora_head="<your_lora_head_name>",
        lora_rank=8,
        lora_dropout=32,
        lora_dropout=0.1
    ),
)
# 创建RAG工作流
rag_worlflow=SimpleRAG_Workflow(**config)


## 运行

In [ ]:
gen_kwargs=dict(max_new_tokens=2048,batch_size=4)
prompt_prefix="请依据所给的问题以及文本材料，生成最合适的答案。\n\n问题:\n{query}\n\n文本材料:\n{evidences}\n\n"
qas=load_from_json("data\\qas.json")
results=rag_worlflow.query(prompt_prefix,[n['问题'] for n in qas[:10]],10,**gen_kwargs)
results